# Model Evaluation
Evaluate models using NRMSE with reproducible splits and generate plots.

In [ ]:
# Setup
import os, sys
import numpy as np
import pandas as pd
import joblib
sys.path.append('..')
from src.evaluation.metrics import nrmse
from src.evaluation.validation import simple_train_test_split, k_fold_split
from src.evaluation.visualization import plot_predictions, plot_residuals
data_path = '../data/processed/features_train.csv'
assert os.path.exists(data_path), f'Missing {data_path}. Run feature engineering first.'

## Load Data and Create Split

In [ ]:
df = pd.read_csv(data_path)
target_col = 'Total_Cooling_Load'
num_df = df.select_dtypes(include=[np.number])
X = num_df.drop(columns=[target_col]).fillna(num_df.median(numeric_only=True))
y = num_df[target_col].values
X_train, X_test, y_train, y_test = simple_train_test_split(X.values, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

## Load Trained Models (if available)

In [ ]:
models = {}
# XGBoost
xgb_path = '../models/trained_models/xgboost_model.pkl'
if os.path.exists(xgb_path):
    models['xgboost'] = joblib.load(xgb_path)
# LightGBM
lgb_path = '../models/trained_models/lightgbm_model.pkl'
if os.path.exists(lgb_path):
    models['lightgbm'] = joblib.load(lgb_path)
# LSTM
lstm_path = '../models/trained_models/lstm_model.h5'
try:
    if os.path.exists(lstm_path):
        from tensorflow.keras.models import load_model
        models['lstm'] = load_model(lstm_path)
except Exception as e:
    print('Skipping LSTM load:', e)
sorted(list(models.keys()))

## Evaluate Models (NRMSE)

In [ ]:
results = {}
for name, model in models.items():
    if name == 'lstm':
        X_test_in = np.expand_dims(X_test, axis=1)
        y_pred = model.predict(X_test_in, verbose=0).ravel()
    else:
        y_pred = model.predict(X_test)
    results[name] = float(nrmse(y_test, y_pred))
results

## Visualization

In [ ]:
# Produce and save plots for available models
for name, model in models.items():
    if name == 'lstm':
        X_in = np.expand_dims(X_test, axis=1)
        y_pred = model.predict(X_in, verbose=0).ravel()
    else:
        y_pred = model.predict(X_test)
    plot_predictions(y_test, y_pred, save_path=f'../reports/figures/{name}_predictions.png')
    plot_residuals(y_test, y_pred, save_path=f'../reports/figures/{name}_residuals.png')
'Saved plots to reports/figures'

## Write Summary to reports/model_performance.md

In [ ]:
summary_path = '../reports/model_performance.md'
lines = ['# Model Performance (Evaluation)', '', '| Model | NRMSE |', '|---|---|']
for k, v in sorted(results.items()):
    lines.append(f'| {k} | {v:.5f} |')
content = '\n'.join(lines)
with open(summary_path, 'w') as f: f.write(content)
summary_path